## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import numpy as np
A = np.random.randint(0, 1000, size=(1000000,))
A

array([243, 586, 107, ..., 983, 813, 533])

In [2]:
def f1(A):
    acc, cnt=0,0
    for ai in A:
        acc+=(ai+100)
        cnt+=1
    return acc/cnt

In [3]:
%timeit f1(A)

563 ms ± 87.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def f2(A):
    acc=0
    for ai in A:
        acc+=(ai+100)
    return acc/len(A)

In [5]:
%timeit f2(A)

471 ms ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%lprun -f f2 f2(A)

In [7]:
!pip install line_profiler

You should consider upgrading via the 'c:\users\admin\pycharmprojects\ml_course\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [10]:
%load_ext line_profiler

In [12]:
def f3(A):
    acc=0
    for ai in A:
        acc+=ai
    return acc/len(A)+100

In [13]:
%timeit f3(A)

533 ms ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
def f4(A):
    return A.mean()+100
%timeit f4(A)

2.52 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
import numba

In [16]:
@numba.njit
def f5(A):
    acc, cnt=0,0
    for ai in A:
        acc+=(ai+100)
        cnt+=1
    return acc/cnt

In [17]:
f5(A)

599.461498

In [18]:
%timeit f5(A)

1.36 ms ± 79.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [19]:
import pandas as pd

df=pd.DataFrame(np.random.randint(0, 1000, size=(2000000, 4)), columns=['col1', 'col2', 'col3', 'col4'])
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v', 'w', 'x', 'y', 'z']
df['key']= np.random.choice(letters, 2000000, replace=True)
df

,col1,col2,col3,col4,key
0,309,412,783,504,x
1,385,820,851,547,p
2,912,450,435,965,z
3,997,812,708,724,e
4,954,283,112,90,z
...,...,...,...,...,...
1999995,308,921,146,693,m
1999996,692,480,245,687,x
1999997,995,931,795,221,u
1999998,89,865,805,970,s


In [20]:
def g1(df):
    letters=['a', 'b', 'c', 'd', 'e']
    dfs=[]
    for letter in letters:
        my_df=df[df['key']==letter]
        dfs.append(my_df)
    return pd.concat(dfs,axis=0)

In [21]:
def g2(df):
    letters=['a', 'b', 'c', 'd', 'e']
    dfs=[]
    for letter in letters:
        my_df=df[df['key'].isin(letters)]
        dfs.append(my_df)
    return pd.concat(dfs,axis=0)

In [22]:
%timeit g1(df)

1.46 s ± 54.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit g2(df)

1.19 s ± 89.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%time g1(df)

Wall time: 1.47 s


,col1,col2,col3,col4,key
13,145,757,493,308,a
34,326,582,889,334,a
55,120,132,973,523,a
68,809,15,541,825,a
133,38,651,581,220,a
...,...,...,...,...,...
1999841,682,739,743,999,e
1999847,103,820,441,986,e
1999852,960,645,840,136,e
1999883,393,747,561,260,e


In [25]:
%time g2(df)

Wall time: 1.28 s


,col1,col2,col3,col4,key
3,997,812,708,724,e
7,8,53,660,873,c
9,774,613,251,524,d
13,145,757,493,308,a
20,21,421,965,680,c
...,...,...,...,...,...
1999970,250,366,593,367,b
1999972,916,664,274,848,b
1999973,971,686,61,447,d
1999975,814,925,344,103,b


In [26]:
%%time 
g1(df)

Wall time: 1.45 s


,col1,col2,col3,col4,key
13,145,757,493,308,a
34,326,582,889,334,a
55,120,132,973,523,a
68,809,15,541,825,a
133,38,651,581,220,a
...,...,...,...,...,...
1999841,682,739,743,999,e
1999847,103,820,441,986,e
1999852,960,645,840,136,e
1999883,393,747,561,260,e


In [27]:
%%time 
g2(df)

Wall time: 1.03 s


,col1,col2,col3,col4,key
3,997,812,708,724,e
7,8,53,660,873,c
9,774,613,251,524,d
13,145,757,493,308,a
20,21,421,965,680,c
...,...,...,...,...,...
1999970,250,366,593,367,b
1999972,916,664,274,848,b
1999973,971,686,61,447,d
1999975,814,925,344,103,b


In [25]:
%lprun -f g1 g1(df)

In [26]:
%lprun -f g2 g2(df)

## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [28]:
recipes = pd.read_csv('recipes_sample.csv',sep=',', index_col=1)
recipes

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...
267661,zurie s holey rustic olive and cheddar bread,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
386977,zwetschgenkuchen bavarian plum cake,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
103312,zwiebelkuchen southwest german onion cake,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN


In [37]:
reviews = pd.read_csv('reviews_sample.csv',sep=',', index_col=0)
reviews.index.names = ['id']
reviews

,user_id,recipe_id,date,rating,review
id,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."


In [38]:
def fun1(reviews):
    summa=0
    cnt=0
    for row in reviews.iterrows():
        if (row[1]['date'] < '2011-01-01') and (row[1]['date'] > '2009-12-31'):
            summa+=row[1]['rating']
            cnt+=1
    return summa/cnt   
fun1(reviews)

4.4544402182900615

In [39]:
%time fun1(reviews)

Wall time: 12.5 s


4.4544402182900615

In [41]:
def fun2(reviews):
    summa=0
    reviews = reviews.loc[(reviews['date'] < '2011-01-01') & (reviews['date'] > '2009-12-31')]
    for row in reviews.iterrows():
        summa+=row[1]['rating']
    return summa/len(reviews)
fun2(reviews)

4.4544402182900615

In [42]:
%time fun2(reviews)

Wall time: 861 ms


4.4544402182900615

In [32]:
def fun3(reviews):
    reviews = reviews.loc[(reviews['date'] < '2011-01-01') & (reviews['date'] > '2009-12-31')]
    return reviews['rating'].mean()
fun3(reviews)

4.4544402182900615

In [33]:
%time fun3(reviews)

Wall time: 29.7 ms


4.4544402182900615

In [34]:
%lprun -f fun3 fun3(reviews)

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [36]:
def fun4(reviews):
    reviews = reviews.loc[(reviews['date'] < '2011-01-01') & (reviews['date'] > '2009-12-31')]
    return reviews['rating'].sum()/len(reviews)
fun4(reviews)

4.4544402182900615

In [38]:
%time fun4(reviews)

Wall time: 26.9 ms


4.4544402182900615

In [39]:
%lprun -f fun4 fun4(reviews)

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [41]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [44]:
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [45]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

In [83]:
def get_word_reviews_count(df):
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews_count:
                word_reviews_count[word] = 0
            word_reviews_count[word]+=1
            
    return word_reviews_count

In [84]:
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) ля расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта.
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт

среднего значения рейтинга по всем отзывам для этого рецепта

In [68]:
mn = reviews[reviews['rating'] != 0].groupby('recipe_id')['rating'].mean()
mn

recipe_id
48        2.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536360    5.000000
536473    5.000000
536547    5.000000
536728    4.000000
536729    4.750000
Name: rating, Length: 27440, dtype: float64

In [71]:
reviews4 = reviews[reviews['rating'] != 0].merge(mn, on="recipe_id", how='inner')
reviews4

,user_id,recipe_id,date,rating_x,review,rating_y
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,4.818182
1,86359,57993,2003-08-10,5,My sisters husband caught a salmon off the Que...,4.818182
2,67026,57993,2005-06-11,5,I enjoyed this very much. Turned out pretty d...,4.818182
3,371697,57993,2006-10-27,5,Delicious! Perfect blend of flavors. I didn'...,4.818182
4,175286,57993,2004-12-29,5,Delicious! Made this last night with relative...,4.818182
...,...,...,...,...,...,...
119886,157617,94096,2004-08-14,5,wheeee!!! \r\nincredible!!!!\r\n;)\r\ntook a l...,5.000000
119887,1207345,368208,2009-07-13,2,"I found this to be a good dish,but not great.I...",2.000000
119888,357323,154964,2008-10-31,5,I was searching to see if this recipe had been...,5.000000
119889,160974,187418,2007-10-15,5,Update: I loved this so much that I had to eat...,5.000000


In [80]:
((reviews4['rating_y'] - reviews4['rating_x']) / reviews4['rating_y']).abs().sum() * 100 / len(reviews4)

7.951710005387515

In [81]:
def mape(reviews):
    mn = reviews[
        reviews['rating'] != 0
    ].groupby('recipe_id')['rating'].mean()
    
    reviews4 = reviews[
        reviews['rating'] != 0
    ].merge(mn, on="recipe_id", how='inner')
    
    return ((reviews4['rating_y'] - reviews4['rating_x']) 
            / reviews4['rating_y']).abs().sum() * 100 / len(reviews4)

In [82]:
%lprun -f mape mape(reviews)

In [86]:
def mape(reviews):
    mn = reviews[
        reviews['rating'] != 0
    ].groupby('recipe_id')['rating'].mean()
    
    reviews4 = reviews[
        reviews['rating'] != 0
    ].merge(mn, on="recipe_id", how='inner')
    
    return ((reviews4['rating_y'] - reviews4['rating_x']) 
            / reviews4['rating_y']).abs().sum() * 100 / len(reviews4)